In [1]:
import ROOT
import numpy as np
from array import array

Welcome to JupyROOT 6.22/08


In [2]:
#data to read
f_mc = ROOT.TFile.Open("OG_data/BdToJpsiKstar_BMuonFilter_v0.root", "READ")
f_data = ROOT.TFile.Open("OG_data/Data-BParking_v0.root", "READ")

tree_data = f_data.Get("demo/ntupleTree")
tree_mc = f_mc.Get("demo/ntupleTree")

f_data.ls()
f_data.Print()

print("The loaded MC tree 'ntupleTree' contains %d events" % tree_mc.GetEntries())
print("The loaded data tree 'ntupleTree' contains %d events" % tree_data.GetEntries())

# Data histos
histo_B_mass = ROOT.TH1F("Mass of B0", ";GeV;" , 50, 4, 7)
histo_K_star = ROOT.TH1F("Mass of K*",  ";GeV;", 30, 0.6, 1.3)

# MC histos
histo_B_mass_MC = ROOT.TH1F("Mass of B0", ";GeV;" , 50, 4, 7)
histo_K_star_MC = ROOT.TH1F("Mass of K*",  ";GeV;", 30, 0.6, 1.3)

mass_K = 0.493677 #GeV
mass_pi = 0.139571 #GeV
mass_Kstar = 0.89166 #GeV
mass_mu = 0.1056583745 #GeV

Kstar_MC = np.zeros(tree_mc.GetEntries(), dtype= float)


The loaded MC tree 'ntupleTree' contains 33027 events
The loaded data tree 'ntupleTree' contains 943 events
TFile**		OG_data/Data-BParking_v0.root	
 TFile*		OG_data/Data-BParking_v0.root	
  TDirectoryFile*		demo	demo
   OBJ: TTree	ntupleTree	ntupleTree : 0 at: 0x83e8370
   KEY: TTree	ntupleTree;1	ntupleTree
  KEY: TDirectoryFile	demo;1	demo
TFile: name=OG_data/Data-BParking_v0.root, title=, option=READ
******************************************************************************
*Tree    :ntupleTree: ntupleTree                                             *
*Entries :      943 : Total =         2648330 bytes  File  Size =     242574 *
*        :          : Tree compression factor =  10.97                       *
******************************************************************************
*Br    0 :hltNames  : vector<string>                                         *
*Entries :      943 : Total  Size=    1873892 bytes  File Size  =      96075 *
*Baskets :       59 : Basket Size=      3

Warning in <TFile::Append>: Replacing existing TH1: Mass of B0 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: Mass of K* (Potential memory leak).


### Add Kstar branch to TTrees
Data sample

In [4]:
f_data = ROOT.TFile("OG_data/Data-BParking_v0.root", "READ")
tree_data = f_data.Get("demo/ntupleTree")
Kstar_Mass = array('f', [0.])
B_Mass = array('f', [0.])
B_Eta = array('f', [0.])
newFile = ROOT.TFile("data_ntuple.root","RECREATE")
newTree = tree_data.CloneTree(0)
newTree.Branch("Kstar_Mass", Kstar_Mass, "Kstar_Mass/F")
newTree.Branch("B_Mass", B_Mass, "B_Mass/F")
newTree.Branch("B_Eta", B_Eta, "B_Eta/F")

count = 0

for entry in tree_data:
    tlv_muon1 = ROOT.TLorentzVector()
    tlv_muon1.SetPtEtaPhiM(entry.Muon1_pT, entry.Muon1_eta, entry.Muon1_phi, mass_mu)
    tlv_muon2 = ROOT.TLorentzVector()
    tlv_muon2.SetPtEtaPhiM(entry.Muon2_pT, entry.Muon2_eta, entry.Muon2_phi, mass_mu)

    #hypothesis 1: track1 kaon, track2 pi
    tlv1 = ROOT.TLorentzVector()
    tlv1.SetPtEtaPhiM(entry.track1_pT, entry.track1_eta, entry.track1_phi, mass_K)
    tlv2 = ROOT.TLorentzVector()
    tlv2.SetPtEtaPhiM(entry.track2_pT, entry.track2_eta, entry.track2_phi, mass_pi)
    mass_h1 = (tlv1+tlv2).M()

    #hypothesis 2: track1 pion, track2 kaon
    tlv3 = ROOT.TLorentzVector()
    tlv3.SetPtEtaPhiM(entry.track1_pT, entry.track1_eta, entry.track1_phi, mass_pi)
    tlv4 = ROOT.TLorentzVector()
    tlv4.SetPtEtaPhiM(entry.track2_pT, entry.track2_eta, entry.track2_phi, mass_K)
    mass_h2 = (tlv3+tlv4).M()

    #Check which hypothesis is correct for each entry and use it to determine the mass of the Kstar and the B
    diff = np.abs(mass_h1-mass_Kstar)
    diff2 = np.abs(mass_h2-mass_Kstar)

    if (diff < diff2):
        K_star_vector = tlv1+tlv2
    else:
        K_star_vector = tlv3+tlv4
    Kstar_Mass[0]=K_star_vector.M()

    if (((tlv_muon1+tlv_muon2).M()>3 and (tlv_muon1+tlv_muon2).M()<3.15) and (K_star_vector.M()>0.8 and K_star_vector.M()<1) and (entry.Dimuon_CosAlpha>0.9)):
        B_vector = K_star_vector+tlv_muon1+tlv_muon2
        B_Mass[0] = B_vector.M()
        B_Eta[0] = B_vector.Eta()
        
    newTree.Fill()
    count += 1

newTree.Write("", ROOT.TObject.kOverwrite)
newFile.Close()


MC sample

In [5]:
f_mc = ROOT.TFile("OG_data/BdToJpsiKstar_BMuonFilter_v0.root", "READ")
tree_mc = f_mc.Get("demo/ntupleTree")
Kstar_Mass_mc = array('f', [0.])
B_Mass_mc = array('f', [0.])
B_Eta_mc = array('f', [0.])
newFile_mc = ROOT.TFile("MC_ntuple.root","RECREATE")
newTree_mc = tree_mc.CloneTree(0)
newTree_mc.Branch("Kstar_Mass", Kstar_Mass_mc, "Kstar_Mass/F")
newTree_mc.Branch("B_Mass", B_Mass_mc, "B_Mass/F")
newTree_mc.Branch("B_Eta", B_Eta_mc, "B_Eta/F")
count = 0

for entry in tree_mc:
    tlv_muon1 = ROOT.TLorentzVector()
    tlv_muon1.SetPtEtaPhiM(entry.Muon1_pT, entry.Muon1_eta, entry.Muon1_phi, mass_mu)
    tlv_muon2 = ROOT.TLorentzVector()
    tlv_muon2.SetPtEtaPhiM(entry.Muon2_pT, entry.Muon2_eta, entry.Muon2_phi, mass_mu)

    #hypothesis 1: track1 kaon, track2 pi
    tlv1 = ROOT.TLorentzVector()
    tlv1.SetPtEtaPhiM(entry.track1_pT, entry.track1_eta, entry.track1_phi, mass_K)
    tlv2 = ROOT.TLorentzVector()
    tlv2.SetPtEtaPhiM(entry.track2_pT, entry.track2_eta, entry.track2_phi, mass_pi)
    mass_h1 = (tlv1+tlv2).M()

    #hypothesis 2: track1 pion, track2 kaon
    tlv3 = ROOT.TLorentzVector()
    tlv3.SetPtEtaPhiM(entry.track1_pT, entry.track1_eta, entry.track1_phi, mass_pi)
    tlv4 = ROOT.TLorentzVector()
    tlv4.SetPtEtaPhiM(entry.track2_pT, entry.track2_eta, entry.track2_phi, mass_K)
    mass_h2 = (tlv3+tlv4).M()

    #Check which hypothesis is correct for each entry and use it to determine the mass of the Kstar and the B
    diff = np.abs(mass_h1-mass_Kstar)
    diff2 = np.abs(mass_h2-mass_Kstar)

    if (diff < diff2):
        K_star_vector = tlv1+tlv2
    else:
        K_star_vector = tlv3+tlv4
        
    Kstar_Mass_mc[0]=K_star_vector.M()

    if (((tlv_muon1+tlv_muon2).M()>3 and (tlv_muon1+tlv_muon2).M()<3.15) and (K_star_vector.M()>0.8 and K_star_vector.M()<1) and (entry.Dimuon_CosAlpha>0.9)):
        B_vector = K_star_vector+tlv_muon1+tlv_muon2
        B_Mass_mc[0] = B_vector.M()
        B_Eta_mc[0] = B_vector.Eta()

        
    print (B_Mass_mc)
    newTree_mc.Fill()
    count += 1

newTree_mc.Write("", ROOT.TObject.kOverwrite)
newFile_mc.Close()


array('f', [5.253001689910889])
array('f', [5.284503936767578])
array('f', [5.217092037200928])
array('f', [5.23912239074707])
array('f', [5.23718786239624])
array('f', [5.2848615646362305])
array('f', [5.2848615646362305])
array('f', [5.2848615646362305])
array('f', [5.2848615646362305])
array('f', [5.593547344207764])
array('f', [5.272170066833496])
array('f', [5.272170066833496])
array('f', [5.272170066833496])
array('f', [5.311577320098877])
array('f', [5.288171768188477])
array('f', [5.288171768188477])
array('f', [5.278435707092285])
array('f', [5.278435707092285])
array('f', [5.278435707092285])
array('f', [5.278435707092285])
array('f', [5.230533123016357])
array('f', [5.230533123016357])
array('f', [5.230533123016357])
array('f', [5.2751007080078125])
array('f', [5.1444091796875])
array('f', [5.1444091796875])
array('f', [5.2549662590026855])
array('f', [4.837645530700684])
array('f', [4.837645530700684])
array('f', [4.837645530700684])
array('f', [4.837645530700684])
array('f

array('f', [5.2696757316589355])
array('f', [5.2696757316589355])
array('f', [5.2696757316589355])
array('f', [5.258444786071777])
array('f', [4.85349178314209])
array('f', [4.85349178314209])
array('f', [5.298634052276611])
array('f', [5.356781482696533])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.259698867797852])
array('f', [5.296573162078857])
array('f', [5.296573162078857])
array('f', [5.346208572387695])
array('f', [5.339804172515869])
array('f', [5.239803314208984])
array('f', [5.239803314208984])
array('f', [5.239803314208984])
array('f', [5.239803314208984])
array('f', [5.288186550140381])
array('f', [4.903565406799316])
array('f', [4.903565406799316])
array('f', [4.903565406799316])
array('f', [5.279056549072266])
array('f', [5.279056549072266])
array('f', [5.279056549072266])
array('

array('f', [5.291516304016113])
array('f', [5.243834018707275])
array('f', [5.285091876983643])
array('f', [5.285091876983643])
array('f', [5.285091876983643])
array('f', [5.285091876983643])
array('f', [4.938807487487793])
array('f', [4.938807487487793])
array('f', [5.2988667488098145])
array('f', [5.2988667488098145])
array('f', [5.2988667488098145])
array('f', [5.2988667488098145])
array('f', [5.218954086303711])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.253555774688721])
array('f', [5.276645660400391])
array('f', [5.878421306610107])
array('f', [5.878421306610107])
array('f', [5.2680463790893555])
array('f', [5.301868915557861])
array('f', [5.301868915557861])
array('f', [5.2437238693237305])
array('f', [5.198295593261719])
array('f', [5.198295593261719])
array('f', [5.179924964904785])
ar

array('f', [5.750397682189941])
array('f', [5.321855545043945])
array('f', [5.321855545043945])
array('f', [5.321855545043945])
array('f', [5.321855545043945])
array('f', [5.259571075439453])
array('f', [5.483248233795166])
array('f', [5.483248233795166])
array('f', [5.278468608856201])
array('f', [5.300832271575928])
array('f', [5.251664638519287])
array('f', [5.251664638519287])
array('f', [5.336680889129639])
array('f', [5.278402805328369])
array('f', [5.311554431915283])
array('f', [5.145847797393799])
array('f', [5.264237880706787])
array('f', [5.225834369659424])
array('f', [5.225834369659424])
array('f', [5.225834369659424])
array('f', [5.321584701538086])
array('f', [5.2862138748168945])
array('f', [5.2862138748168945])
array('f', [5.2862138748168945])
array('f', [5.221254348754883])
array('f', [5.32143497467041])
array('f', [5.32143497467041])
array('f', [5.185890197753906])
array('f', [5.296631336212158])
array('f', [5.313178062438965])
array('f', [5.33721399307251])
array('f

array('f', [4.931100368499756])
array('f', [5.371922492980957])
array('f', [5.371922492980957])
array('f', [5.226849555969238])
array('f', [5.226849555969238])
array('f', [5.226849555969238])
array('f', [5.226849555969238])
array('f', [5.236255168914795])
array('f', [5.297656059265137])
array('f', [5.297656059265137])
array('f', [5.297656059265137])
array('f', [5.246957778930664])
array('f', [5.246957778930664])
array('f', [5.246957778930664])
array('f', [5.264769077301025])
array('f', [5.270975589752197])
array('f', [5.270975589752197])
array('f', [5.270975589752197])
array('f', [5.270975589752197])
array('f', [5.270975589752197])
array('f', [5.3250603675842285])
array('f', [5.053535461425781])
array('f', [5.053535461425781])
array('f', [5.053535461425781])
array('f', [5.3039069175720215])
array('f', [5.3039069175720215])
array('f', [5.295883655548096])
array('f', [5.248131275177002])
array('f', [5.277571678161621])
array('f', [5.277571678161621])
array('f', [5.277571678161621])
array

...

array('f', [5.216447353363037])
array('f', [5.216447353363037])
array('f', [5.216447353363037])
array('f', [5.216447353363037])
array('f', [5.313094139099121])
array('f', [5.270944118499756])
array('f', [5.270944118499756])
array('f', [5.264174461364746])
array('f', [5.264174461364746])
array('f', [5.288852214813232])
array('f', [5.163013458251953])
array('f', [5.295642375946045])
array('f', [5.295642375946045])
array('f', [5.4465861320495605])
array('f', [5.30330228805542])
array('f', [5.1382598876953125])
array('f', [5.1382598876953125])
array('f', [5.1382598876953125])
array('f', [5.1382598876953125])
array('f', [5.2737836837768555])
array('f', [5.2737836837768555])
array('f', [5.252750873565674])
array('f', [5.252750873565674])
array('f', [5.212825298309326])
array('f', [5.212825298309326])
array('f', [5.307823657989502])
array('f', [5.307823657989502])
array('f', [5.307823657989502])
array('f', [5.310838222503662])
array('f', [5.310838222503662])
array('f', [5.272712230682373])
ar

array('f', [5.242547035217285])
array('f', [5.268227577209473])
array('f', [5.268227577209473])
array('f', [5.268227577209473])
array('f', [5.3137617111206055])
array('f', [5.238409996032715])
array('f', [5.289098739624023])
array('f', [5.244105339050293])
array('f', [5.1219353675842285])
array('f', [5.1219353675842285])
array('f', [5.315486431121826])
array('f', [4.977453708648682])
array('f', [5.275684833526611])
array('f', [5.275684833526611])
array('f', [5.195273399353027])
array('f', [5.195273399353027])
array('f', [5.195273399353027])
array('f', [5.292537212371826])
array('f', [5.292537212371826])
array('f', [5.337734699249268])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.306252956390381])
array('f', [5.276917457580566])
array('f', [5.276917457580566])
array

array('f', [5.263710975646973])
array('f', [5.263710975646973])
array('f', [5.283476829528809])
array('f', [5.372180461883545])
array('f', [5.372180461883545])
array('f', [5.372180461883545])
array('f', [5.274928092956543])
array('f', [5.274928092956543])
array('f', [5.274928092956543])
array('f', [5.296252250671387])
array('f', [5.231961727142334])
array('f', [5.231961727142334])
array('f', [5.231961727142334])
array('f', [5.231961727142334])
array('f', [5.145153522491455])
array('f', [5.145153522491455])
array('f', [5.145153522491455])
array('f', [5.145153522491455])
array('f', [5.278173446655273])
array('f', [5.278173446655273])
array('f', [5.161862373352051])
array('f', [5.197221755981445])
array('f', [5.197221755981445])
array('f', [5.247361660003662])
array('f', [5.24101448059082])
array('f', [5.32639741897583])
array('f', [5.32639741897583])
array('f', [5.32639741897583])
array('f', [5.32639741897583])
array('f', [5.2807536125183105])
array('f', [5.2807536125183105])
array('f', 

array('f', [5.2749481201171875])
array('f', [5.294971466064453])
array('f', [5.294971466064453])
array('f', [5.294971466064453])
array('f', [5.247318267822266])
array('f', [5.238638877868652])
array('f', [5.238638877868652])
array('f', [5.238638877868652])
array('f', [5.238638877868652])
array('f', [5.318300724029541])
array('f', [5.318300724029541])
array('f', [5.318300724029541])
array('f', [5.318300724029541])
array('f', [5.318300724029541])
array('f', [5.220941066741943])
array('f', [5.268768787384033])
array('f', [5.268768787384033])
array('f', [5.268768787384033])
array('f', [5.268768787384033])
array('f', [5.268768787384033])
array('f', [5.306491374969482])
array('f', [5.306491374969482])
array('f', [5.248598575592041])
array('f', [5.8208160400390625])
array('f', [5.287327766418457])
array('f', [5.287327766418457])
array('f', [5.214435577392578])
array('f', [5.214435577392578])
array('f', [5.2558274269104])
array('f', [5.248232841491699])
array('f', [5.248232841491699])
array('f

array('f', [5.282355785369873])
array('f', [5.268128395080566])
array('f', [5.537606716156006])
array('f', [5.537606716156006])
array('f', [5.313119411468506])
array('f', [5.313119411468506])
array('f', [5.29048490524292])
array('f', [5.309083461761475])
array('f', [5.375863075256348])
array('f', [5.295679092407227])
array('f', [5.295679092407227])
array('f', [5.326746463775635])
array('f', [5.326746463775635])
array('f', [5.326746463775635])
array('f', [5.324592113494873])
array('f', [5.283666610717773])
array('f', [5.283666610717773])
array('f', [5.242941856384277])
array('f', [5.242941856384277])
array('f', [5.242941856384277])
array('f', [5.2640910148620605])
array('f', [5.19253396987915])
array('f', [5.34342098236084])
array('f', [5.34342098236084])
array('f', [5.34342098236084])
array('f', [5.34342098236084])
array('f', [5.313404083251953])
array('f', [5.064055442810059])
array('f', [5.202266216278076])
array('f', [5.375180244445801])
array('f', [5.280282020568848])
array('f', [4

In [6]:
# f_mc = ROOT.TFile.Open("BdToJpsiKstar_BMuonFilter_v0.root", "READ")
f = ROOT.TFile.Open("new_data.root", "READ")
tree = f.Get("ntupleTree")

f.ls()
f.Print()

c1 = ROOT.TCanvas("c1", "Title", 800, 600)
tree.Draw("B_Eta")
c1.Draw()

for entry in tree:
    print (entry.B_Eta)

ReferenceError: attempt to access a null-pointer

Error in <TFile::TFile>: file new_data.root does not exist
